In [1]:
import numpy as np
import pandas as pd
import re
import collections
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv(r"C:\Users\YUE\Desktop\interface2.csv",dtype={'Reviews':str})
new_data=data['Reviews']

In [3]:
#简单预处理
def clean_text(text):
    text= text.replace('\n'," ") #用空格代替新的行
    text=re.sub(r"-"," ",text)  #把用“-”连接的单词分开 （close-up分成close和 up）
    pure_text=''
    for letter in text:        #去除字母和空格
        letter=letter.lower()
        if letter.isalpha() or letter==' ':
            pure_text+=letter
    text=' '.join(word for word in pure_text.split() if len(word)>1)
    return text

In [4]:
#错词纠正算法
def token(text):
    
    return re.findall('[a-z]+', text.lower())

path=r'C:\Users\YUE\Desktop\big.txt'
with open(path) as f:
    WORDS=token(f.read())
WORD_COUNTS=collections.Counter(WORDS)

def known(words):
    return {w for w in words if w in WORD_COUNTS}

def edits0(word):
    return {word}


def edits1(word):
    alphabet='abcdefghijklmnopqrstuvwxyz'
    
    def splits(word): 
        return [(word[:i],word[i:]) for i in range(len(word)+1)]
    
    pairs=splits(word)
    deletes=[a+b[1:] for (a,b) in pairs if b]
    transposes=[a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b)>1]
    replaces=[a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts=[a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes+ transposes+ replaces+ inserts)

def edits2(word):
    return{e2 for e1 in edits1(word) for e2 in edits1(e1)}

def correct(word):
    candidates=(known(edits0(word)) or
                known(edits1(word)) or
                known(edits2(word)) or
                [word] )
    return max(candidates,key=WORD_COUNTS.get)

def correct_match(match):
    word=match.group()
    
    def case_of(text):
        return (str.upper if text.isupper() else
                str.lower if text.islower() else
                str.title if text.istitle() else
                str )
    return case_of(word)(correct(word.lower()))


def correct_text_generic(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

In [5]:
#错词纠正
new_df=new_data.copy()
docs=new_df.apply(lambda s:clean_text(s))
doclist=docs.values
reviews_1=[]
for sentence in doclist:
    raw_sentence=nltk.word_tokenize(sentence)
    for word in sentence:
        word=correct_text_generic(word)
    reviews_1.append(raw_sentence)
#print(reviews_1)

In [6]:
#词性还原算法
def get_word_pos(tag):
#     if tag.startswith('J'):
#         return wordnet.ADJ
    if tag.startswith('V'):
        return wordnet.VERB
#     elif tag.startswith('N'):
#         return wordnet.NOUN
#     elif tag.startswith('R'):
#         return wordnet.ADV
    else:
        return None

In [7]:
#词性还原
reviews_2=[]
for sentence in reviews_1:
    tagged_sent=pos_tag(sentence)
    wnl=WordNetLemmatizer()
    lemmas_sent=[]
    for tag in tagged_sent:
        wordnet_pos=get_word_pos(tag[1]) or wordnet.NOUN
        lemmas_sent.append(wnl.lemmatize(tag[0],pos=wordnet_pos))
    reviews_2.append(lemmas_sent)
#print(reviews_2)

In [8]:
#停止词去除
def remove_stopword():
    stopword = []
    with open(r'C:\Users\YUE\Desktop\english.txt', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.replace('\n', '')
            stopword.append(line)
    return stopword

my_stoplst = ["app", "good", "excellent", "please", "they", "very", "too", 
                  "yeah", "amazing", "lovely", "perfect", "much", "best", "yup", "super", "thank",
                  "great",
                  "really", "omg", "gud", "yes", "cool", "fine", "hello", "alright", "poor", "plz", "pls", "google",
                  "facebook",
                  "three", "ones", "one", "two", "five", "four", "old", "new", "asap", "version", "times", "update",
                  "star",
                  "first",
                  "rid", "bit", "annoying", "beautiful", "dear", "master", "evernote", "per", "line", "oh", "ah",
                  "cannot", "doesnt", "won't", "dont", "unless", "you're", "aren't", "i'd", "can't", "wouldn't",
                  "around",
                  "i've", "i'll", "gonna", "ago", "you'll", "you'd", "28th", "gen", "it'll", "vice", "would've",
                  "wasn't",
                  "year", "boy", "they'd",
                  "isnt", "1st", "i'm","im", "nobody", "youtube", "isn't", "don't", "2016", "2017", "since", "near", "god","video",
                   "none","hi","ive","didnt","af","io"]

stopset=list(stopwords.words('english'))

In [9]:
stop_words=stopset+my_stoplst
texts=[[word for word in doc if word not in stop_words] for doc in reviews_2]

In [10]:
for line in texts:
    for words in line:
        print(words,end=' ')
    print('\n')

horrible graphic get big phone screen stay wonder ever get approve release even development 

love ebay active buyer thing nt like way feedback create would like see fee back item could help decide want buy item sow seller feedback page see item number well nothing bad design thing feedback ebay bleahh 

interface allow easy buy sell use desktop site 

everyday save go computer every often shame get user interface 

layout result image item small see option alter layout better use online 

many item ask seller ship cost get also layout get worse 

ebay change layout today wo nt work even uninstall reinstall like tot break 

older interface better search feature lock randomly black screen set friendly search option nt stay set set every search use phone know ebayer sign rate option store download older 

use recent buggy get crap together ebay website look like program h kid maybe take web design class something 

layout color scheme annoyance use uninstalling newest reinstall previous 